## Multiple Tools

This notebook demonstrates how to build a LangGraph agent with **multiple tools** that can intelligently choose which tool to use based on the user's query.

## Key Concepts

### Tools Used
- **ArxivQueryRun**: Searches academic papers on Arxiv
- **WikipediaQueryRun**: Searches Wikipedia articles
- **date_time_tool**: Custom tool that returns the current date and time

### Architecture
The graph follows a **ReAct pattern** (Reasoning + Acting):
1. **Chatbot Node**: The LLM receives messages and decides whether to use tools
2. **Tool Node**: Executes the requested tools
3. **Conditional Edge**: `tools_condition` checks if the LLM made a tool call
   - If yes → route to Tool Node
   - If no → end the conversation

### Key Components
- **`ToolNode`**: Pre-built node that executes tool calls from the LLM's response
- **`tools_condition`**: Conditional edge that checks if `tool_calls` exist in the last message
- **`bind_tools()`**: Attaches tools to the LLM so it knows what tools are available

The LLM automatically selects the appropriate tool based on the query context!

In [6]:
import os
from typing import TypedDict, Literal, Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

load_dotenv()

OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL")

In [10]:
llm = ChatOllama(model="glm-5:cloud", base_url=OLLAMA_BASE_URL) # "glm-5:cloud" is a cloud model that has access to tools.

In [7]:
from langchain_core.messages import AnyMessage

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [8]:
from langchain_core.tools import tool
from datetime import datetime

@tool
def date_time_tool() -> str:
    """Use this tool to get the current date and time."""
    return datetime.now().isoformat()

In [9]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

# Create tools with API wrappers
arxiv_tool = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())
wiki_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [arxiv_tool, wiki_tool, date_time_tool]

In [11]:
llm_with_tools=llm.bind_tools(tools)

In [12]:
from langgraph.prebuilt import ToolNode, tools_condition

# Create the tool node
tool_node = ToolNode(tools)

# Define the chatbot function
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build the graph
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")

graph = graph_builder.compile()

In [14]:
messages=graph.invoke({"messages":HumanMessage(content="1706.03762")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

1706.03762
================================== Ai Message ==================================

I'll look up that arXiv paper for you.
Tool Calls:
  arxiv (f5a34ed4-8608-47f7-9f28-3feeda6b94eb)
 Call ID: f5a34ed4-8608-47f7-9f28-3feeda6b94eb
  Args:
    query: 1706.03762
================================= Tool Message =================================
Name: arxiv

Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurr

In [15]:
messages=graph.invoke({"messages":HumanMessage(content="What is AI?")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

What is AI?
================================== Ai Message ==================================
Tool Calls:
  wikipedia (00b04ed8-58d7-4ca8-995f-311a499c3a64)
 Call ID: 00b04ed8-58d7-4ca8-995f-311a499c3a64
  Args:
    query: Artificial Intelligence AI
================================= Tool Message =================================
Name: wikipedia

Page: A.I. Artificial Intelligence
Summary: A.I. Artificial Intelligence (or simply A.I.) is a 2001 science fiction drama film directed by Steven Spielberg. The screenplay by Spielberg and screen story by Ian Watson are loosely based on the 1969 short story "Supertoys Last All Summer Long" by Brian Aldiss. Set in a futuristic society, the film stars Haley Joel Osment as David, a childlike android uniquely programmed with the ability to love. Jude Law, Frances O'Connor, Brendan Gleeson and William Hurt star in supporting roles.
Development of A.I. originally began a

In [16]:
messages=graph.invoke({"messages":HumanMessage(content="What is the date today?")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

What is the date today?
================================== Ai Message ==================================

I'll get the current date and time for you.
Tool Calls:
  date_time_tool (7129ccd0-a9d7-4c0d-a989-60bd6ed0d125)
 Call ID: 7129ccd0-a9d7-4c0d-a989-60bd6ed0d125
  Args:
================================= Tool Message =================================
Name: date_time_tool

2026-02-22T23:02:44.624946
================================== Ai Message ==================================

Today's date is **February 22, 2026**. 

The current time is 11:02 PM (23:02).


In [17]:
messages=graph.invoke({"messages":HumanMessage(content="What is the date after 2 months?")})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

What is the date after 2 months?
================================== Ai Message ==================================

I'll get the current date and then calculate the date after 2 months for you.
Tool Calls:
  date_time_tool (2c9fba85-3141-42c6-9ca9-a1e247a73098)
 Call ID: 2c9fba85-3141-42c6-9ca9-a1e247a73098
  Args:
================================= Tool Message =================================
Name: date_time_tool

2026-02-22T23:03:16.218102
================================== Ai Message ==================================

The current date is February 22, 2026.

Two months from now would be **April 22, 2026**.
